# Error Mining

Le but de ce notebook est de trouver les erreurs les plus fréquentes du parseur afin de corriger le treebank en ayant le plus d'impact possible. Nous regarderons donc quelles sont les types d'erreurs faites par le parseur, quelles sont les plus problématiques et quelle distribution ont ces erreurs.

In [ ]:
PATH_TREEBANK = "../data/sud_naija-NSC.with_prediction.conllu"

In [ ]:
# le fichier est t-il bien présent ?
try:
    with open(PATH_TREEBANK, "r") as f:
        print("Le fichier est présent au chemin : {}".format(PATH_TREEBANK))
        pass
except FileNotFoundError:
    print("Le fichier n'est PAS , revoyez le chemin : {}".format(PATH_TREEBANK))
    exit()

In [ ]:
# On peut commencer à parser le fichier avec la librairie conllup
from conllup.conllup import readConlluFile 
sentences = readConlluFile(PATH_TREEBANK)

if len(sentences) == 0:
    raise ValueError("Le fichier est vide !")
else :
    print("Le fichier contient {} phrases".format(len(sentences)))

## Influence de la taille de la phrase

In [ ]:
# On récupère les nombres de réussite et d'échec pour chaque longueur de phrase
results_per_length = {}
def get_default_result():
    return {
            "total_token": 0,
            "success_upos": 0, 
            "success_deprel": 0, 
            "success_head": 0, 
            "success_head_and_deprel": 0, 
        }

for sentence in sentences:
    tokens = sentence["treeJson"]["nodesJson"].values()
    length = len(tokens)
    if length not in results_per_length:
        results_per_length[length] = get_default_result()
    
    for token in tokens:
        head_token_ID = str(token['HEAD'])
        deprel = token['DEPREL']
        upos = token['UPOS']

        head_pred = token['MISC']['head_pred']
        deprel_pred = token['MISC']['deprel_pred']
        upos_pred = token['MISC']['upos_pred']

        if head_token_ID == head_pred:
            results_per_length[length]["success_head"] += 1

        if deprel == deprel_pred:
            results_per_length[length]["success_deprel"] += 1
        
        if head_token_ID == head_pred and deprel == deprel_pred:
            results_per_length[length]["success_head_and_deprel"] += 1

        if upos == upos_pred:
            results_per_length[length]["success_upos"] += 1

        results_per_length[length]["total_token"] += 1

# On affiche les résultats
import json
print(json.dumps(results_per_length, indent=4))

In [ ]:
# Nous utilisons matplotlib pour faire faire nos graphiques
import matplotlib as mpl
import matplotlib.pyplot as plt

# Voici des paramètres pour que les graphiques soient plus lisibles, changez les à votre convenance
params = {
   'axes.labelsize': 8,
 #  'text.fontsize': 8,
   'legend.fontsize': 10,
   'xtick.labelsize': 10,
   'ytick.labelsize': 10,
   'text.usetex': False,
   'figure.figsize': [10, 5]
   }
mpl.rcParams.update(params)
plt.style.use('seaborn-v0_8-darkgrid') # pour avoir un fond gris quadrillé

In [ ]:
# Essayons de mettre tout ça dans un graphique
# On va plotter les ratio d'echec (1 - ratio_reussite) ça sera plus visuel
# Dans un même graphique on va plotter les 3 ratios : upos, deprel, head

colors = ["red", "green", "blue", "orange"]

x_lim = [5, 60] # on ne prend pas en compte les phrases de moins de 5 tokens et de plus de 80 tokens

plt.figure(figsize=(12,8))

for i, label in enumerate(["deprel", "upos", "head"]):
    x = list(range(x_lim[0], x_lim[1]))
    y = []
    for length in x:
        number_success = results_per_length.get(length, get_default_result())["success_{}".format(label)]
        number_total = results_per_length.get(length, get_default_result())["total_token"]
        
        fail_ratio = 0
        if number_total != 0:
            fail_ratio = (number_total - number_success) / number_total
        
        y.append(fail_ratio)

    plt.plot(range(x_lim[0], x_lim[1]), y, label=label, color=colors[i])

    # si on veut afficher les approximations linéaires :
    from sklearn.linear_model import LinearRegression
    import numpy as np
    regressor = LinearRegression()  
    x = np.array(x).reshape(-1, 1)
    y = np.array(y).reshape(-1, 1)
    regressor.fit(x, y)
    plt.plot(x, regressor.predict(x) ,color=colors[i], linestyle="dotted")

plt.xlabel("taille de la phrase")
plt.ylabel("probabilité d'erreur pour un token")
plt.legend()
plt.xlim([x_lim[0]+1,x_lim[1]])


- Est-ce que ces résultats étaient attendus ?
- À quel(s) biais devons nous faire attention ?

NB : Il faudrait utiliser des tests de significativité afin de prouver que les corrélations linéaires ne soient pas dû au hasard

## Erreurs de prédiction

In [ ]:
# On récupère d'abord les listes des features réelles et prédites
gold_uposs = []
gold_deprels = []
gold_heads = []

pred_uposs = []
pred_deprels = []
pred_heads = []

for sentence in sentences:
    tokens = sentence["treeJson"]["nodesJson"].values()
    for token in tokens:
        gold_uposs.append(token['UPOS'])
        gold_deprels.append(token['DEPREL'])
        gold_heads.append(token['HEAD'])

        pred_uposs.append(token['MISC']['upos_pred'])
        pred_deprels.append(token['MISC']['deprel_pred'])
        pred_heads.append(token['MISC']['head_pred'])

set_uposs = set(gold_uposs + pred_uposs)
set_deprels = set(gold_deprels + pred_deprels)

## Erreurs de POS

In [ ]:
# Quelles sont les plus grosses confusions de POS ?

# On va créer un dictionnaire qui va contenir pour chaque couple de POS, le nombre de fois où on a eu cette confusion
confusion_matrix_upos = {}
sorted_set_uposs = sorted(set_uposs)

for gold_upos in sorted_set_uposs:
    confusion_matrix_upos[gold_upos] = {}
    for pred_upos in sorted_set_uposs:
        confusion_matrix_upos[gold_upos][pred_upos] = 0

for pred_upos, gold_upos in zip(pred_uposs, gold_uposs):
    confusion_matrix_upos[gold_upos][pred_upos] += 1

# On affiche le dictionnaire
print(json.dumps(confusion_matrix_upos, indent=4))



# 

In [ ]:
# Nous allons d'abord afficher le nombre de relation par pair de catégories du discours
# Il est important de faire cela avant d'afficher le ratio, car le ratio ne veut rien dire si le nombre de relation est trop faible

import pandas as pd
import seaborn as sns
# Créer un DataFrame pandas pour stocker les données
upos_df = pd.DataFrame(confusion_matrix_upos)
confusion_matrix = upos_df.groupby(upos_df.columns, axis=1).sum().astype(int)

# Créer la matrice de confusion avec seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, cmap="Blues", fmt='d')

# Ajouter les titres et labels
plt.title('Matrice de confusion (UPOS)')
plt.xlabel('Prédits')
plt.ylabel('Réels')

# Afficher la matrice de confusion
plt.show()

- Combien d'adjectifs ont été annotés comme des noms ?
- Quelle est la catégorie la plus annotée ?
- Quelle est la catégorie la mieux prédite ?

## Erreurs des deprels

In [ ]:
# Quelles sont les plus grosses confusions de deprel ?

# On va créer un dictionnaire qui va contenir pour chaque couple de deprel, le nombre de fois où on a eu cette confusion
confusion_matrix_deprel = {}
sorted_set_deprels = sorted(set_deprels)

for gold_deprel in sorted_set_deprels:
    confusion_matrix_deprel[gold_deprel] = {}
    for pred_deprel in sorted_set_deprels:
        confusion_matrix_deprel[gold_deprel][pred_deprel] = 0

for pred_deprel, gold_deprel in zip(pred_deprels, gold_deprels):
    confusion_matrix_deprel[gold_deprel][pred_deprel] += 1

# On affiche le dictionnaire
print(json.dumps(confusion_matrix_deprel, indent=4))



In [ ]:
# Nous allons d'abord afficher le nombre de relation par pair de catégories du discours
# Il est important de faire cela avant d'afficher le ratio, car le ratio ne veut rien dire si le nombre de relation est trop faible

import pandas as pd
import seaborn as sns
# Créer un DataFrame pandas pour stocker les données
deprel_df = pd.DataFrame(confusion_matrix_deprel)


# # Gardons uniquement les relations qui contiennent "comp"
deprels_to_keep = [deprel for deprel in sorted(set_deprels) if "mod" in deprel]
deprel_df = deprel_df.loc[deprels_to_keep, deprels_to_keep]


# Créer la matrice de confusion avec seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(deprel_df, annot=True, cmap="Blues", fmt='d')

# Ajouter les titres et labels
plt.title('Matrice de confusion (deprel)')
plt.xlabel('Prédits')
plt.ylabel('Réels')

# Afficher la matrice de confusion
plt.show()